In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import os
import re

In [2]:
client_id = "zuo60iNcVNsA5BmyfHL64nzVMAfZ-suiFONibf8_EjOPXeUQgrIqX-w3CA-RRji8"
client_secret = "KsD2On2pG2zR7vgvTLkp7zW5i3LHoKuIB8OqyN0uV-AAIocMwwi8JQTeaolFozKSvQpiQkH6V0wGh6Jr7qSW1w"
token = "_Q2Mi2ftYFYIkFAUPY9lfkROyNXdj5ML8_6riblV-YCcfHvEvhy5k5Ax5zcWvd-E"
artist_name = 'taylor swift'

### Functions to get urls to song lyrics

In [3]:
def get_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + token}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

def get_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = get_artist_info(artist_name, page)
        json = response.json()
        
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
        
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
                
        if (len(songs) == song_cap):
            break
        else:
            page += 1
    
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs

### Test functions to get urls

In [ ]:
get_song_url(artist_name, 300)

### Function to get lyrics

In [4]:
def get_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    
    title = html.find('title')
    title_text = title.get_text()
    title_text = title_text.encode('ascii', errors='ignore').decode()
    song_title = title_text.replace("TaylorSwift  ", "").replace(" Lyrics | Genius Lyrics", " ")
    lyrics = html.find_all('div', class_="Lyrics__Container-sc-1ynbvzw-7 jjqBBp")
    full_lyrics = ''
    for section in lyrics:
        current_section = section.get_text() 
        full_lyrics = full_lyrics + current_section
    full_lyrics = re.sub(r'[\(\[].*?[\)\]]', '', full_lyrics)
    full_lyrics = os.linesep.join([s for s in full_lyrics.splitlines() if s])
    
    lyric_words = [word for word in full_lyrics.split(" ") if word != ""]
    obj = {}
    obj['song_title'] = song_title.lower()
    obj['lyrics'] = lyric_words
    return obj
    
#     title_and_lyrics[song_title.lower()] = full_lyrics
#     return title_and_lyrics    

### Test function to get lyrics

In [ ]:
lyrics = get_lyrics('https://genius.com/Taylor-swift-cardigan-lyrics')
print(lyrics)

### Define and run function to get urls + lyrics and write output to JSON file

In [6]:
def write_lyrics_to_file(artist_name, song_count):
    all_songs = []
    urls = get_song_url(artist_name, song_count)
    for url in urls:
        title_and_lyrics = get_lyrics(url)
        all_songs.append(title_and_lyrics)
        
    filename = 'lyrics_taylor_swift.json'
    with open(filename, 'w') as f_obj:
        json.dump(all_songs, f_obj)
        
write_lyrics_to_file(artist_name, 300)

Found 300 songs by taylor swift


### Open file with song titles and lyrics

In [7]:
with open('lyrics_taylor_swift.json', 'r') as f:
    song_lyrics_obj = json.load(f)
song_lyrics_obj

[{'song_title': 'cardigan ',
  'lyrics': ['Vintage',
   'tee,',
   'brand',
   'new',
   'phoneHigh',
   'heels',
   'on',
   'cobblestonesWhen',
   'you',
   'are',
   'young,',
   'they',
   'assume',
   'you',
   'know\u2005nothingSequin\u2005smile,',
   'black',
   'lipstickSensual\u2005politicsWhen',
   'you',
   'are',
   'young,',
   'they',
   'assume\u2005you',
   'know',
   'nothingBut',
   'I',
   'knew',
   "youDancin'",
   'in',
   'your',
   "Levi'sDrunk",
   'under',
   'a',
   'streetlight,',
   'II',
   'knew',
   'youHand',
   'under',
   'my',
   'sweatshirtBaby,',
   'kiss',
   'it',
   'better,',
   'IAnd',
   'when',
   'I',
   'felt',
   'like',
   'I',
   'was',
   'an',
   'old',
   'cardiganUnder',
   "someone's",
   'bedYou',
   'put',
   'me',
   'on',
   'and',
   'said',
   'I',
   'was',
   'your',
   'favoriteA',
   'friend',
   'to',
   'all',
   'is',
   'a',
   'friend',
   'to',
   'noneChase',
   'two',
   'girls,',
   'lose',
   'the',
   'oneWhen'

### Construct dataframe of song titles and lyrics

In [8]:
lyrics_df = pd.DataFrame(data=song_lyrics_obj)
lyrics_df

,song_title,lyrics
0,cardigan,"[Vintage, tee,, brand, new, phoneHigh, heels, ..."
1,exile,"[I, can, see, you, standing,, honeyWith, his, ..."
2,lover,"[We, could, leave, the, Christmas, lights, up,..."
3,the 1,"[I'm, doing, good,, I'm, on, some, new, shitBe..."
4,look what you made me do,"[I, don't, like, your, little, gamesDon't, lik..."
...,...,...
295,check out this view,"[I, don’t, need, a, lotBut, there's, one, thin..."
296,my cure,"[I, know, we've, got, a, lot, to, sayBetween, ..."
297,sunshine,"[I, walk, in, the, room, there's, a, beautiful..."
298,i forgot that you existed (piano/vocal),"[So, the, clip, that, you're, about, to, hearI..."


### Save dataframe to CSV file for storage in PostgreSQL database.

In [9]:
lyrics_df.to_csv('lyrics_df.csv',index=False)